# Save signal maps

In [8]:
import os
import numpy as np
import pandas as pd
import rasterio as rio
import matplotlib.pyplot as plt
import xarray as xr
import datetime as dt
import rioxarray
import seaborn as sns
import geopandas as gpd

In [9]:
# function to rewrite coordinates from metadata
def coord_range(ds):
    latrange = np.linspace(float(ds.attrs['Y_FIRST']),
                           ((float(ds.attrs['Y_STEP'])*float(ds.attrs['LENGTH']))+float(ds.attrs['Y_FIRST'])),
                           int(ds.attrs['LENGTH']))
    lonrange = np.linspace(float(ds.attrs['X_FIRST']),
                           ((float(ds.attrs['X_STEP'])*float(ds.attrs['WIDTH']))+float(ds.attrs['X_FIRST'])),
                           int(ds.attrs['WIDTH']))
    return latrange, lonrange

def mintpy2d_to_xarray(fn, crs):
    ds = xr.open_dataset(fn)
    ds = ds.rename_dims({'phony_dim_0':'y',
                         'phony_dim_1':'x'
                        })
    latrange, lonrange = coord_range(ds)
    ds = ds.assign_coords({'y': ('y', latrange),
                           'x': ('x', lonrange)})
    ds = ds.rio.write_crs(crs)
    
    return ds

In [10]:
orbit_list = ['AT151', 'DT56']
year_list = ['2017', '2018', '2019', '2020', '2021', '2022']
frame_list = ['frame_1', 'frame_2', 'frame_3']

In [15]:
# function to clip files and run mintpy for multiple years 
def save_veloc_multiyear(orbit_list, year_list, frame_list, crs=32613):
    # hardcoded paths for now 
    home_path = '/mnt/d/indennt/'
    for orbit in orbit_list:
        for frame in frame_list:
            for year in year_list:
                print(f'working on {orbit}, {frame}, {year}')
                mintpy_path = f'{home_path}/signal_mintpy/{orbit}/{frame}/mintpy_{year}'
    
                vel_ds = mintpy2d_to_xarray(f'{mintpy_path}/velocity.h5', crs)
                vel_ds = vel_ds.velocity.rio.write_nodata(0, inplace=True)
                #vel_ds.rio.to_raster(f'{home_path}/signal_maps/{orbit}/{frame}_{year}_signal.tif')
    
                corr_ds = mintpy2d_to_xarray(f'{mintpy_path}/avgSpatialCoh.h5', crs)
                corr_ds = corr_ds.rio.reproject_match(vel_ds)
                corr_ds = corr_ds.coherence.rio.write_nodata(0, inplace=True)
                corr_ds.rio.to_raster(f'{home_path}/signal_maps/{orbit}/{frame}_{year}_corr.tif')

In [16]:
save_veloc_multiyear(orbit_list, year_list, frame_list)

working on AT151, frame_1, 2017
working on AT151, frame_1, 2018
working on AT151, frame_1, 2019
working on AT151, frame_1, 2020
working on AT151, frame_1, 2021
working on AT151, frame_1, 2022
working on AT151, frame_2, 2017
working on AT151, frame_2, 2018
working on AT151, frame_2, 2019
working on AT151, frame_2, 2020
working on AT151, frame_2, 2021
working on AT151, frame_2, 2022
working on AT151, frame_3, 2017
working on AT151, frame_3, 2018
working on AT151, frame_3, 2019
working on AT151, frame_3, 2020
working on AT151, frame_3, 2021
working on AT151, frame_3, 2022
working on DT56, frame_1, 2017
working on DT56, frame_1, 2018
working on DT56, frame_1, 2019
working on DT56, frame_1, 2020
working on DT56, frame_1, 2021
working on DT56, frame_1, 2022
working on DT56, frame_2, 2017
working on DT56, frame_2, 2018
working on DT56, frame_2, 2019
working on DT56, frame_2, 2020
working on DT56, frame_2, 2021
working on DT56, frame_2, 2022
working on DT56, frame_3, 2017
working on DT56, fram